In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [17]:
%load_ext autoreload
%autoreload 2

from os.path import join

import numpy as np

from pyspark.sql import SQLContext

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
path_root = "/home/tlin/notebooks/data"
path_pickle_train = join(path_root, "cache/two_domain/split_data/train")
path_pickle_test = join(path_root, "cache/two_domain/split_data/test")
path_pickle_baseline_sim = join(path_root, "cache/two_domain/item_based_sim/base_sim")
path_pickle_extended_sim = join(path_root, "cache/two_domain/extend_sim/extendsim")

In [8]:
extended_simRDD = sc.pickleFile(path_pickle_extended_sim)

In [23]:
mapping_range = 1
privacy_epsilon = 0.6
sim_method = 'ad_cos'
rpo = 0.1

def calculate_prob(lines, replacement_selection_list):
    """calculate probability (not normalized)."""
    probs = []
    for line in zip(lines, replacement_selection_list):
        tmp = 1.0 * np.exp(
            privacy_epsilon * line[0][1] / (
                2 * mapping_range * line[1][1]))
        probs.append((line[0][0], tmp))
    return probs

def normalize_prob(prob_pairs):
    """normalize probability."""
    sum_prob = sum([pair[1] for pair in prob_pairs])
    normalized_prob_pairs = [
        (pair[0], pair[1] / sum_prob) for pair in prob_pairs]
    return normalized_prob_pairs

t = extended_simRDD.take(1)[0]

iid, lines = t
lines = sorted(lines, key=lambda x: - abs(x[1]))[: 10]

replacement_selection_list = [
    (line[0], 1) for line in lines]


prob_pairs = calculate_prob(lines, replacement_selection_list)
normalized_prob_pairs = normalize_prob(prob_pairs)
list(map(lambda l: l[0], normalized_prob_pairs))